# Notebook for fitting the background and asymptotic relation with pca

In [ ]:
#%load_ext autoreload
#%autoreload 2
import os
import pandas as pd
import numpy as np
from asy_bkg_fitting import spectrum_fit 
from matplotlib.pyplot import *
rcParams['font.size'] = 18

In [ ]:
download_dir = '/home/nielsemb/work/mounts/Bluebear_data/data'

workDir = '/home/nielsemb/work/repos/granulation'

prior_data_fname = os.path.join(*[workDir, 'bkgfit_output_nopca.csv']) 
prior_data = pd.read_csv(prior_data_fname)

In [ ]:
i = 3 

ID = prior_data.loc[i, 'ID']
print(ID)

outputDir = os.path.join(*[workDir, 'results', ID])

if not os.path.exists(outputDir):
    os.makedirs(outputDir)

_numax = prior_data.loc[i, 'numax'] # tgt numax
_teff = prior_data.loc[i, 'teff'] # tgt numax
_bp_rp = prior_data.loc[i, 'bp_rp'] # tgt numax
 
obs = {'numax': [10**_numax, 0.01*10**_numax], 
       'teff': [10**_teff, 100],
       'bp_rp': [_bp_rp, 0.1]} 

sfit = spectrum_fit(ID, obs, download_dir, pcadim=6, N=200, fname=prior_data_fname)

dynSampler, dynSamples = sfit.runDynesty(progress=True)

figM, axM = subplots(figsize=(16,9))
sfit.plotModel(figM, axM, dynSamples, outputDir=outputDir); # Plot model based on posterior samples
axM.set_yscale('linear')
axM.set_xscale('linear')

#sfit.storeResults(outputDir)

In [ ]:
outputDir

In [ ]:
import dill
ext = f'pca{sfit.DR.dims_R}'
gfitpath = os.path.join(*[outputDir, os.path.basename(outputDir) + f'_{ext}.gfit'])
gfitpath

with open(gfitpath, 'wb') as outfile:
    dill.dump(sfit, outfile)

In [ ]:
sfit.DR.erank

In [ ]:
figP, axP = subplots(figsize=(16,9))
sfit.plotModel(figP, axP, outputDir=outputDir); # Plot model from median of the prior
axP.set_yscale('log')
axP.set_xscale('log')
#axP.set_xlim(300, 575)

nu0s = sfit._asymptotic_relation(10**sfit.DR.data_F[0, 7], 
                                 10**sfit.DR.data_F[0, 0], 
                                 sfit.DR.data_F[0, 6], 
                                 10**sfit.DR.data_F[0, 2])
for nu in nu0s:
    axP.axvline(nu)

In [ ]:

#sfit.storeResults(outputDir) # Store the results

In [ ]:

#axM.set_xlim(50, 150)

In [ ]:
# Pick some targets from various parts of the numax range.
test_numaxs = np.linspace(min(prior_data['numax']), max(prior_data['numax']), 4)
idxs = np.array([np.argmin(abs(prior_data['numax'].values - nu)) for nu in test_numaxs])
idxs

In [ ]:
import utils
figM, axM = subplots(figsize=(16,9))
plotModel(sfit, figM, axM, dynSamples); # Plot model based on posterior samples
axM.set_yscale('linear')
axM.set_xscale('linear')
axM.set_xlim(25, 75)

In [ ]:
N = 1
prior_samples = sfit.ptform(np.zeros(sfit.ndim) + 0.5)

theta_asy, theta_bkg, theta_extra = sfit.unpackParams(prior_samples)

In [ ]:
# Pick some targets from various parts of the numax range.
test_numaxs = np.linspace(min(prior_data['numax']), max(prior_data['numax']), 10)
idxs = np.array([np.argmin(abs(prior_data['numax'].values - nu)) for nu in test_numaxs])

# Optional weighting function to plug in, use weights_args dict to set parameters
def wfunc(self, n=1):
     
    ppf, pdf = self.getQuantileFuncs(self.data_F[:, :1])

    w = 1/pdf[0](self.data_F[:, 0])**n
       
    return w

In [ ]:
for i in [8615,  8568, 13325,  6323]:

    ID = prior_data.loc[i, 'ID']
    print(ID)

    outputDir = os.path.join(*[workDir, 'results', ID])

    if not os.path.exists(outputDir):
        os.makedirs(outputDir)

    _numax = prior_data.loc[i, 'numax'] # tgt numax
    _teff = prior_data.loc[i, 'teff'] # tgt numax
    _bp_rp = prior_data.loc[i, 'bp_rp'] # tgt numax

    obs = {'numax': [10**_numax, 0.01*10**_numax], 
           'teff': [10**_teff, 100],
           'bp_rp': [_bp_rp, 0.1]} 

    for j, ndim in enumerate([2, 4, 8, 16]):

        sfit = spectrum_fit(ID, obs, download_dir, pcadim=ndim, N=200, fname=prior_data_fname)

        dynSampler, dynSamples = sfit.runDynesty(progress=True)

        sfit.storeResults(outputDir)

In [ ]:
from scipy.stats import ks_2samp as ks

In [ ]:
ks(hist0[0], hist1[0])

In [ ]:
10**1.46508561520692
